In [1]:
import pandas as pd
import numpy as np
from matplotlib.pyplot import figure, plot
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import requests

pd.set_option('display.max_columns', 200)


In [56]:
from datetime import date, timedelta, datetime

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2022, 4, 30)
end_date = date(2022, 5, 31)
for single_date in daterange(start_date, end_date):
    single_datetime = single_date.strftime("%Y-%m-%d")+"T00:00:00"

In [57]:
def parse_svo(r):
    svo_departure = pd.DataFrame()

    for i in r["data"]["routes"]:

        try:
            print(i["leg"]["status"], i["flightId"]["carrier"], 
                  i["flightId"]["flightNumber"], 
                  i["operatingBy"]["scheduled"])
        except:
            continue

        if i["leg"]["status"] != 'Arrived':
            continue

        qq = pd.DataFrame([
            i["flightId"]["carrier"],
            i["flightId"]["flightNumber"],
            i["leg"]["equipment"]["aircraft"]["actual"]["type"],
            i["leg"]["equipment"]["aircraft"]["actual"]["registration"],

            # registration
            i["leg"]["transition"]["registration"]["start"]["utc"],
            i["leg"]["transition"]["registration"]["end"]["utc"],

            # boarding
            i["leg"]["transition"]["boarding"]["start"]["utc"],
            i["leg"]["transition"]["boarding"]["end"]["utc"],

            # departure
            i["leg"]["departure"]["latest"]["airportCode"],
            i["leg"]["departure"]["times"].get("actualBlockOff", {}).get("utc", np.NaN),
            i["leg"]["departure"]["times"]["scheduledDeparture"]["utc"],
            i["leg"]["departure"]["times"].get("actualTakeOff", {}).get("utc", np.NaN),

            i["leg"]["departure"].get("gate", np.NaN),
            i["leg"]["departure"]["terminal"],

            # arrival
            i["leg"]["arrival"]["scheduled"]["airportCode"],
            i["leg"]["arrival"]["latest"]["airportCode"],
            i["leg"]["arrival"]["times"]["scheduledArrival"]["utc"],
            i["leg"]["arrival"]["times"].get("actualTouchDown", {}).get("utc", {}),
            i["leg"]["arrival"]["times"].get("deboarding", {}).get("start", {}).get("utc", np.NaN),
            i["leg"]["arrival"]["times"].get("deboarding", {}).get("stop", {}).get("utc", np.NaN),

            # deboarding
            i["leg"]["transition"]["deboarding"]["start"]["utc"],
            i["leg"]["transition"]["deboarding"]["end"]["utc"],

            # status
            i["leg"]["status"],
            i["operatingBy"]["scheduled"]

        ])

        qq = qq.T

        svo_departure = pd.concat([svo_departure, qq]).reset_index().drop(["index"], axis=1)
        
    svo_departure.columns = ["carrier", "flight_number", 
                   "aircraft_type", "aircraft_registration",
                   "registration_start", "registration_end", 
                   "boarding_start", "boarding_end",
                   "departure_airportCode", "departure_actualBlockOff",
                   "departure_scheduledDeparture", "departure_actualTakeOff",
                   "departure_gate", "departure_terminal", 
                   "arrival_scheduled_airportCode", "arrival_latest_airportCode",
                   "arrival_scheduledArrival", "arrival_actualTouchDown", 
                   "arrival_deboardingStart", "arrival_deboardingStop",
                   "deboarding_start", "deboarding_end",
                   "status", "operatingBy"
                  ]
    return svo_departure

In [58]:
start_date = date(2022, 4, 29)
end_date = date(2022, 7, 2)

dep = pd.DataFrame()

for single_date in daterange(start_date, end_date):
    print(single_date)
    single_datetime = single_date.strftime("%Y-%m-%d")+"T00:00:00"
    r = requests.get(f"https://flights.aeroflot.ru/api/flights/v1.1/ru/board?type=onlineboard&departure=SVO&dateFrom={single_datetime}&dateTo={single_datetime}&timeFrom=00:00:00&timeTo=23:59:59&returnTo=23:59:59",
                 headers= {
                     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36 OPR/51.0.2830.55"
                 }).json()
    df = parse_svo(r)
    dep = pd.concat([dep, df]).reset_index().drop(["index"], axis=1)
    

2022-04-29
Cancelled SU 6275 SU
Arrived SU 1966 SU
Arrived SU 1478 SU
Arrived SU 1524 SU
Arrived SU 1244 SU
Arrived SU 1428 SU
Cancelled SU 6499 SU
Arrived SU 0046 SU
Arrived SU 1646 SU
Cancelled SU 6505 SU
Arrived SU 1306 SU
Arrived SU 6531 FV
Arrived SU 1508 SU
Arrived SU 1562 SU
Arrived SU 1436 SU
Cancelled SU 6425 SU
Cancelled SU 6521 SU
Arrived SU 1522 SU
Arrived SU 1510 SU
Arrived SU 1512 SU
Arrived SU 1110 SU
Arrived SU 0002 SU
Arrived SU 1138 SU
Arrived SU 1382 SU
Arrived SU 6253 FV
Arrived SU 1462 SU
Arrived SU 1602 SU
Arrived SU 1246 SU
Arrived SU 1516 SU
Arrived SU 6489 FV
Arrived SU 1000 SU
Arrived SU 6257 FV
Arrived SU 0006 SU
Arrived SU 1188 SU
Cancelled SU 6299 SU
Arrived SU 1320 SU
Cancelled SU 6477 SU
Arrived SU 1444 SU
Arrived SU 1134 SU
Arrived SU 6433 FV
Arrived SU 6377 FV
Arrived SU 1486 SU
Arrived SU 1886 SU
Arrived SU 1304 SU
Cancelled SU 6067 SU
Arrived SU 1210 SU
Arrived SU 1332 SU
Cancelled SU 6337 SU
Cancelled SU 6527 SU
Arrived SU 1498 SU
Arrived SU 0010 SU


Arrived SU 6275 FV
Arrived SU 1966 SU
Arrived SU 1524 SU
Arrived SU 1478 SU
Cancelled SU 6499 SU
Arrived SU 1244 SU
Arrived SU 0046 SU
Arrived SU 1428 SU
Arrived SU 1894 SU
Cancelled SU 6505 SU
Arrived SU 1306 SU
Arrived SU 6531 FV
Arrived SU 1382 SU
Arrived SU 1508 SU
Arrived SU 1512 SU
Arrived SU 1982 SU
Arrived SU 1436 SU
Arrived SU 6425 FV
Arrived SU 1562 SU
Arrived SU 1522 SU
Arrived SU 6521 FV
Arrived SU 1110 SU
Arrived SU 1130 SU
Arrived SU 1138 SU
Arrived SU 0002 SU
Arrived SU 6253 FV
Arrived SU 1602 SU
Arrived SU 1462 SU
Arrived SU 1246 SU
Arrived SU 1516 SU
Arrived SU 6489 FV
Arrived SU 1000 SU
Arrived SU 6257 FV
Landed SU 6429 FV
Arrived SU 6299 FV
Arrived SU 1320 SU
Cancelled SU 6477 SU
Arrived SU 1444 SU
Arrived SU 1134 SU
Arrived SU 6433 FV
Arrived SU 1886 SU
InFlight SU 6377 FV
Arrived SU 1304 SU
Arrived SU 1486 SU
Arrived SU 1520 SU
Arrived SU 1014 SU
Arrived SU 1052 SU
Arrived SU 1210 SU
Cancelled SU 6067 SU
Arrived SU 1332 SU
Cancelled SU 6337 SU
Arrived SU 0010 SU
Ar

Arrived SU 0024 SU
Arrived SU 1710 SU
Arrived SU 1186 SU
Arrived SU 1414 SU
Arrived SU 1440 SU
Arrived SU 6261 FV
Arrived SU 1242 SU
Arrived SU 6483 FV
Arrived SU 6251 FV
Arrived SU 1730 SU
Arrived SU 1704 SU
Arrived SU 1006 SU
Arrived SU 1832 SU
Arrived SU 1718 SU
Arrived SU 6349 FV
Cancelled SU 6287 SU
Cancelled SU 6993 SU
Cancelled SU 6495 SU
Arrived SU 0026 SU
Arrived SU 1140 SU
Cancelled SU 6399 SU
Arrived SU 1194 SU
Arrived SU 1750 SU
Arrived SU 1532 SU
Arrived SU 0028 SU
Cancelled SU 6255 SU
Cancelled SU 6285 SU
Arrived SU 1594 SU
Arrived SU 6289 FV
Arrived SU 1008 SU
Arrived SU 0030 SU
Arrived SU 1322 SU
Arrived SU 6235 FV
Arrived SU 1548 SU
Arrived SU 1702 SU
Arrived SU 1394 SU
Arrived SU 1130 SU
Arrived SU 1480 SU
Arrived SU 1334 SU
Arrived SU 0032 SU
Arrived SU 1314 SU
Arrived SU 1260 SU
Arrived SU 1288 SU
Cancelled SU 1714 SU
Arrived SU 1442 SU
Arrived SU 1172 SU
Arrived SU 1406 SU
Arrived SU 6381 FV
Arrived SU 1882 SU
Arrived SU 6247 FV
Cancelled SU 6503 SU
Arrived SU 6199

Cancelled SU 6469 SU
Arrived SU 1214 SU
Arrived SU 0024 SU
Arrived SU 6283 FV
Arrived SU 1710 SU
Arrived SU 1440 SU
Arrived SU 1414 SU
Arrived SU 6421 FV
Arrived SU 6261 FV
Arrived SU 1242 SU
Cancelled SU 6483 SU
Cancelled SU 6251 SU
Arrived SU 1006 SU
Arrived SU 6349 FV
Arrived SU 1718 SU
Arrived SU 1704 SU
Cancelled SU 6287 SU
Cancelled SU 6993 SU
Arrived SU 0026 SU
Cancelled SU 6495 SU
Arrived SU 1842 SU
Arrived SU 1140 SU
Cancelled SU 6399 SU
Arrived SU 1532 SU
Arrived SU 1750 SU
Arrived SU 6197 FV
Cancelled SU 6255 SU
Arrived SU 6289 FV
Arrived SU 1008 SU
Arrived SU 1480 SU
Arrived SU 1322 SU
Cancelled SU 6235 SU
Arrived SU 1548 SU
Arrived SU 1702 SU
Arrived SU 0030 SU
Arrived SU 1394 SU
Arrived SU 1130 SU
Arrived SU 1898 SU
Arrived SU 1334 SU
Arrived SU 1260 SU
Arrived SU 0032 SU
Arrived SU 1314 SU
Arrived SU 1288 SU
Arrived SU 6301 FV
Arrived SU 1442 SU
Arrived SU 1406 SU
Arrived SU 1172 SU
Arrived SU 6381 FV
Arrived SU 1882 SU
Arrived SU 6247 FV
Cancelled SU 6503 SU
Arrived SU 

Arrived SU 1434 SU
Arrived SU 1768 SU
Cancelled SU 6069 SU
Cancelled SU 6393 SU
Arrived SU 1324 SU
Arrived SU 1058 SU
Arrived SU 6229 FV
Arrived SU 1404 SU
Arrived SU 0022 SU
Arrived SU 1248 SU
Arrived SU 1700 SU
Arrived SU 1330 SU
Cancelled SU 6335 SU
Arrived SU 0038 SU
Arrived SU 1302 SU
Arrived SU 1010 SU
Arrived SU 6491 FV
Arrived SU 1118 SU
Arrived SU 6379 FV
Arrived SU 1730 SU
Arrived SU 6121 FV
Arrived SU 6283 FV
Arrived SU 0024 SU
Arrived SU 1710 SU
Arrived SU 1440 SU
Arrived SU 6421 FV
Arrived SU 1414 SU
Arrived SU 6261 FV
Arrived SU 6483 FV
Arrived SU 1242 SU
Cancelled SU 6251 SU
Arrived SU 1704 SU
Arrived SU 6349 FV
Arrived SU 1006 SU
Arrived SU 1832 SU
Arrived SU 1718 SU
Cancelled SU 6287 SU
Cancelled SU 6993 SU
Arrived SU 0026 SU
Cancelled SU 6495 SU
Arrived SU 1140 SU
Cancelled SU 6399 SU
Arrived SU 1194 SU
Arrived SU 1532 SU
Arrived SU 2134 SU
Scheduled SU 2138 SU
Arrived SU 0028 SU
Arrived SU 6289 FV
Arrived SU 1008 SU
Arrived SU 1480 SU
Arrived SU 1322 SU
Arrived SU 00

...

Cancelled SU 6465 SU
Arrived SU 1366 SU
Arrived SU 1460 SU
Arrived SU 1780 SU
Arrived SU 1190 SU
Arrived SU 1758 SU
Arrived SU 1230 SU
Arrived SU 1416 SU
Arrived SU 1544 SU
Arrived SU 6387 FV
Arrived SU 1518 SU
Arrived SU 1892 SU
Arrived SU 0012 SU
Arrived SU 1610 SU
Cancelled SU 6225 SU
Cancelled SU 6457 SU
Arrived SU 6539 FV
Arrived SU 1136 SU
Cancelled SU 6913 SU
Arrived SU 1312 SU
Arrived SU 1220 SU
Cancelled SU 6153 SU
Arrived SU 1002 SU
Arrived SU 1218 SU
Cancelled SU 6239 SU
Arrived SU 6991 FV
Arrived SU 1122 SU
Arrived SU 0014 SU
Cancelled SU 6497 SU
Arrived SU 2146 SU
Arrived SU 1488 SU
Arrived SU 6075 FV
Arrived SU 1362 SU
Arrived SU 1320 SU
Cancelled SU 6479 SU
Arrived SU 1456 SU
Arrived SU 1068 SU
Arrived SU 1420 SU
Arrived SU 6603 FV
Arrived SU 1202 SU
Arrived SU 0042 SU
Arrived SU 1234 SU
Arrived SU 1552 SU
Cancelled SU 6063 SU
Arrived SU 6893 FV
Cancelled SU 6409 SU
Arrived SU 1500 SU
Arrived SU 0016 SU
Arrived SU 6391 FV
Arrived SU 1696 SU
Arrived SU 6415 FV
Cancelled S

Arrived SU 1204 SU
Cancelled SU 6421 SU
Arrived SU 6389 FV
Arrived SU 6261 FV
Arrived SU 6615 FV
Cancelled SU 6483 SU
Arrived SU 1316 SU
Cancelled SU 6251 SU
Arrived SU 1704 SU
Arrived SU 1006 SU
Arrived SU 6349 FV
Cancelled SU 6155 SU
Arrived SU 1832 SU
Arrived SU 1718 SU
Cancelled SU 6287 SU
Arrived SU 0068 SU
Cancelled SU 6495 SU
Arrived SU 1604 SU
Arrived SU 0026 SU
Arrived SU 1418 SU
Cancelled SU 6399 SU
Arrived SU 1140 SU
Arrived SU 1750 SU
Arrived SU 1194 SU
Arrived SU 1532 SU
Arrived SU 1734 SU
Cancelled SU 1726 SU
Arrived SU 6219 FV
Cancelled SU 6255 SU
Arrived SU 0028 SU
Cancelled SU 6285 SU
Arrived SU 1008 SU
Cancelled SU 6883 SU
Cancelled SU 6273 SU
Arrived SU 0030 SU
Arrived SU 1742 SU
Arrived SU 6235 FV
Arrived SU 1548 SU
Arrived SU 1394 SU
Arrived SU 1782 SU
Arrived SU 6485 FV
Arrived SU 1702 SU
Arrived SU 1334 SU
Arrived SU 0032 SU
Arrived SU 1260 SU
Arrived SU 1714 SU
Landed SU 1314 SU
Arrived SU 6601 FV
Cancelled SU 6301 SU
Arrived SU 6993 FV
Arrived SU 1442 SU
Arrive

2022-06-28
Arrived SU 1264 SU
Arrived SU 6275 FV
Arrived SU 1196 SU
Arrived SU 1478 SU
Arrived SU 1524 SU
Arrived SU 6609 FV
Arrived SU 6551 FV
InFlight SU 1430 SU
Cancelled SU 6499 SU
Arrived SU 1428 SU
Arrived SU 1638 SU
Arrived SU 1424 SU
Arrived SU 0046 SU
Arrived SU 1382 SU
Arrived SU 1482 SU
Arrived SU 1306 SU
Cancelled SU 6505 SU
Arrived SU 6531 FV
Landed SU 1562 SU
Arrived SU 1508 SU
Arrived SU 0036 SU
Arrived SU 1512 SU
Arrived SU 1656 SU
Arrived SU 1110 SU
Arrived SU 1472 SU
InFlight SU 1536 SU
Arrived SU 1436 SU
Arrived SU 6533 FV
Arrived SU 2148 SU
Cancelled SU 6425 SU
Arrived SU 1522 SU
Arrived SU 1592 SU
Arrived SU 0002 SU
Arrived SU 6253 FV
Arrived SU 1138 SU
Arrived SU 1640 SU
Arrived SU 1462 SU
Arrived SU 1602 SU
Arrived SU 1516 SU
Cancelled SU 6489 SU
Arrived SU 1668 SU
Arrived SU 1000 SU
Cancelled SU 6123 SU
Arrived SU 1356 SU
Arrived SU 1410 SU
Cancelled SU 6309 SU
Arrived SU 0006 SU
Arrived SU 1270 SU
Cancelled SU 6257 SU
Arrived SU 1254 SU
Arrived SU 1188 SU
Cance

Cancelled SU 6063 SU
Arrived SU 1684 SU
Cancelled SU 6409 SU
Arrived SU 6391 FV
Landed SU 1500 SU
Arrived SU 0016 SU
Cancelled SU 6141 SU
Arrived SU 1696 SU
Landed SU 1402 SU
Arrived SU 0042 SU
InFlight SU 1266 SU
Arrived SU 6891 FV
Cancelled SU 6041 SU
Arrived SU 6527 FV
Arrived SU 1642 SU
Arrived SU 0018 SU
Arrived SU 1124 SU
Arrived SU 1364 SU
Cancelled SU 6511 SU
Arrived SU 1276 SU
Arrived SU 1310 SU
Arrived SU 1176 SU
Arrived SU 1412 SU
Cancelled SU 6271 SU
Arrived SU 0020 SU
Arrived SU 6607 FV
Arrived SU 1212 SU
Arrived SU 1384 SU
Arrived SU 1004 SU
Arrived SU 6079 FV
Arrived SU 1132 SU
Arrived SU 1422 SU
Arrived SU 6359 FV
Arrived SU 6907 FV
Arrived SU 1058 SU
Arrived SU 1128 SU
Arrived SU 1232 SU
Arrived SU 0052 SU
Arrived SU 1842 SU
Arrived SU 6711 FV
Arrived SU 6555 FV
Arrived SU 1300 SU
Arrived SU 1368 SU
Arrived SU 6605 FV
Arrived SU 1680 SU
Cancelled SU 6211 SU
Arrived SU 6535 FV
Arrived SU 1182 SU
Arrived SU 1012 SU
Arrived SU 1226 SU
Arrived SU 1290 SU
Cancelled SU 6069 

Cancelled SU 6399 SU
Arrived SU 1750 SU
Arrived SU 1532 SU
Arrived SU 1604 SU
Arrived SU 0068 SU
Arrived SU 1194 SU
Arrived SU 6219 FV
Arrived SU 1734 SU
Cancelled SU 6255 SU
Cancelled SU 6285 SU
Arrived SU 0028 SU
Arrived SU 6289 FV
Cancelled SU 6883 SU
Arrived SU 1008 SU
Arrived SU 1594 SU
Cancelled SU 6273 SU
Arrived SU 1480 SU
Arrived SU 6235 FV
Arrived SU 1742 SU
Arrived SU 0030 SU
Arrived SU 1548 SU
Arrived SU 1702 SU
Arrived SU 1394 SU
Arrived SU 6485 FV
Arrived SU 1898 SU
Arrived SU 1334 SU
Arrived SU 1260 SU
Arrived SU 0032 SU
Arrived SU 1314 SU
Arrived SU 1714 SU
Cancelled SU 6301 SU
Arrived SU 1442 SU
Arrived SU 1172 SU
Arrived SU 0044 SU
Arrived SU 0320 SU
Arrived SU 6381 FV
Arrived SU 1510 SU
Arrived SU 1406 SU
Arrived SU 1882 SU
Arrived SU 6291 FV
Arrived SU 1250 SU
Arrived SU 1732 SU
Cancelled SU 6503 SU
Cancelled SU 6247 SU
Arrived SU 1198 SU
Arrived SU 1288 SU
Arrived SU 1458 SU
Arrived SU 1762 SU
Arrived SU 1216 SU
Arrived SU 0062 SU
Arrived SU 1396 SU
Arrived SU 1706

In [59]:
dep = dep.sort_values(by='departure_scheduledDeparture')

In [60]:
dep = dep.drop_duplicates(['flight_number', 'departure_scheduledDeparture'])

In [61]:
dep

,carrier,flight_number,aircraft_type,aircraft_registration,registration_start,registration_end,boarding_start,boarding_end,departure_airportCode,departure_actualBlockOff,departure_scheduledDeparture,departure_actualTakeOff,departure_gate,departure_terminal,arrival_scheduled_airportCode,arrival_latest_airportCode,arrival_scheduledArrival,arrival_actualTouchDown,arrival_deboardingStart,arrival_deboardingStop,deboarding_start,deboarding_end,status,operatingBy
13,SU,1510,320,73762,2022-04-27T23:00:00Z,2022-04-28T22:15:00Z,2022-04-28T22:29:00Z,2022-04-28T22:53:00Z,SVO,2022-04-28T23:25:00Z,2022-04-28T18:45:00Z,2022-04-28T23:38:00Z,104,B,SGC,SGC,2022-04-28T22:00:00Z,2022-04-29T02:40:00Z,NaN,NaN,2022-04-29T02:45:00Z,2022-04-29T03:15:00Z,Arrived,SU
6,SU,1646,320,VPBKP,2022-04-27T22:00:00Z,2022-04-28T21:15:00Z,2022-04-28T21:21:00Z,2022-04-28T21:41:00Z,SVO,2022-04-28T21:55:00Z,2022-04-28T19:45:00Z,2022-04-28T22:06:00Z,121,B,NJC,NJC,2022-04-28T23:15:00Z,2022-04-29T00:57:00Z,NaN,NaN,2022-04-29T01:01:00Z,2022-04-29T01:31:00Z,Arrived,SU
0,SU,1966,SU9,89026,2022-04-27T21:10:00Z,2022-04-28T20:25:00Z,2022-04-28T20:33:00Z,2022-04-28T20:54:00Z,SVO,2022-04-28T21:07:00Z,2022-04-28T21:10:00Z,2022-04-28T21:17:00Z,135,C,EVN,EVN,2022-04-29T01:15:00Z,2022-04-29T00:43:00Z,NaN,NaN,2022-04-29T00:47:00Z,2022-04-29T01:17:00Z,Arrived,SU
2,SU,1524,321,73725,2022-04-27T21:10:00Z,2022-04-28T20:25:00Z,2022-04-28T20:31:00Z,2022-04-28T21:02:00Z,SVO,2022-04-28T21:10:00Z,2022-04-28T21:10:00Z,2022-04-28T21:22:00Z,111,B,RGK,RGK,2022-04-29T01:35:00Z,2022-04-29T01:10:00Z,2022-04-28T19:49:00Z,NaN,2022-04-29T01:15:00Z,2022-04-29T01:45:00Z,Arrived,SU
1,SU,1478,320,73732,2022-04-27T21:15:00Z,2022-04-28T20:30:00Z,2022-04-28T20:37:00Z,2022-04-28T21:00:00Z,SVO,2022-04-28T21:08:00Z,2022-04-28T21:15:00Z,2022-04-28T21:19:00Z,110,B,ABA,ABA,2022-04-29T01:50:00Z,2022-04-29T01:18:00Z,NaN,NaN,2022-04-29T01:22:00Z,2022-04-29T01:52:00Z,Arrived,SU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13452,SU,1408,320,73766,2022-06-30T20:45:00Z,2022-07-01T20:00:00Z,2022-07-01T20:07:00Z,2022-07-01T20:32:00Z,SVO,2022-07-01T20:39:00Z,2022-07-01T20:45:00Z,2022-07-01T21:01:00Z,108,B,SVX,SVX,2022-07-01T23:10:00Z,2022-07-01T23:06:00Z,NaN,NaN,2022-07-01T23:11:00Z,2022-07-01T23:41:00Z,Arrived,SU
13455,SU,6523,SU9,89116,2022-06-30T20:50:00Z,2022-07-01T20:05:00Z,2022-07-01T20:08:00Z,2022-07-01T20:28:00Z,SVO,2022-07-01T20:48:00Z,2022-07-01T20:50:00Z,2022-07-01T21:05:00Z,NaN,C,NQZ,NQZ,2022-07-02T00:05:00Z,2022-07-02T00:01:00Z,NaN,NaN,2022-07-02T00:10:00Z,2022-07-02T00:40:00Z,Arrived,FV
13456,SU,6347,SU9,89032,2022-06-30T20:55:00Z,2022-07-01T20:10:00Z,2022-07-01T20:08:00Z,2022-07-01T20:28:00Z,SVO,2022-07-01T20:48:00Z,2022-07-01T20:55:00Z,2022-07-01T21:08:00Z,NaN,B,GOJ,GOJ,2022-07-01T22:10:00Z,2022-07-01T21:58:00Z,NaN,NaN,2022-07-01T22:05:00Z,2022-07-01T22:35:00Z,Arrived,FV
13454,SU,1484,73H,73113,2022-06-30T20:55:00Z,2022-07-01T20:10:00Z,2022-07-01T20:16:00Z,2022-07-01T20:39:00Z,SVO,2022-07-01T20:45:00Z,2022-07-01T20:55:00Z,2022-07-01T21:06:00Z,107,B,KJA,KJA,2022-07-02T01:35:00Z,2022-07-02T01:32:00Z,NaN,NaN,2022-07-02T01:38:00Z,2022-07-02T02:08:00Z,Arrived,SU


In [62]:
dep.to_csv("svo_dep_aeroflot_rossiya_05_06_2022.csv", index=False)

## ARR

In [70]:
def parse_svo(r):
    
    svo_arrival = pd.DataFrame()

    for i in r["data"]["routes"]:

        try:
            print(i["leg"]["status"], i["flightId"]["carrier"], 
                  i["flightId"]["flightNumber"], 
                  i["operatingBy"]["scheduled"])
        except:
            continue

        if i["leg"]["status"] != 'Arrived':
            continue

        qq = pd.DataFrame([
            i["flightId"]["carrier"],
            i["flightId"]["flightNumber"],
            i["leg"]["equipment"]["aircraft"]["actual"]["type"],
            i["leg"]["equipment"]["aircraft"]["actual"]["registration"],

            # registration
            i["leg"]["transition"]["registration"]["start"]["utc"],
            i["leg"]["transition"]["registration"]["end"]["utc"],

            # boarding
            i["leg"]["transition"]["boarding"]["start"]["utc"],
            i["leg"]["transition"]["boarding"]["end"]["utc"],

            # departure
            i["leg"]["departure"]["latest"]["airportCode"],
            i["leg"]["departure"]["times"].get("actualBlockOff", {}).get("utc", np.NaN),
            i["leg"]["departure"]["times"]["scheduledDeparture"]["utc"],
            i["leg"]["departure"]["times"].get("actualTakeOff", {}).get("utc", np.NaN),

            # arrival
            i["leg"]["arrival"].get("gate", np.NaN),
            i["leg"]["arrival"]["scheduled"]["airportCode"],
            i["leg"]["arrival"]["latest"]["airportCode"],
            i["leg"]["arrival"]["terminal"],

            i["leg"]["arrival"]["times"]["scheduledArrival"]["utc"],
            i["leg"]["arrival"]["times"].get("actualTouchDown", {}).get("utc", np.NaN),
            i["leg"]["arrival"]["times"].get("deboarding", {}).get("start", {}).get("utc", np.NaN),
            i["leg"]["arrival"]["times"].get("deboarding", {}).get("stop", {}).get("utc", np.NaN),

            # deboarding
            i["leg"]["transition"]["deboarding"]["start"]["utc"],
            i["leg"]["transition"]["deboarding"]["end"]["utc"],

            # status
            i["leg"]["status"],
            i["operatingBy"]["scheduled"]

        ])

        qq = qq.T

        svo_arrival = pd.concat([svo_arrival, qq]).reset_index().drop(["index"], axis=1)
    svo_arrival.columns = ["carrier", "flight_number", 
                       "aircraft_type", "aircraft_registration",
                       "registration_start", "registration_end", 
                       "boarding_start", "boarding_end",
                       "departure_airportCode", "departure_actualBlockOff",
                       "departure_scheduledDeparture", "departure_actualTakeOff",
                       "arrival_gate", "arrival_scheduled", 
                       "arrival_latest", "arrival_terminal",
                       "arrival_scheduledArrival", "arrival_actualTouchDown", 
                       "arrival_deboardingStart", "arrival_deboardingStop",
                       "deboarding_start", "deboarding_end",
                       "status", "operatingBy"
                      ]
    return svo_arrival

In [71]:
start_date = date(2022, 4, 29)
end_date = date(2022, 7, 2)

arr = pd.DataFrame()

for single_date in daterange(start_date, end_date):
    print(single_date)
    single_datetime = single_date.strftime("%Y-%m-%d")+"T00:00:00"
    r = requests.get(f"https://flights.aeroflot.ru/api/flights/v1.1/ru/board?type=onlineboard&arrival=SVO&dateFrom={single_datetime}&dateTo={single_datetime}&timeFrom=00:00:00&timeTo=23:59:59&returnTo=23:59:59",
                 headers= {
                     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36 OPR/51.0.2830.55"
                 }).json()
    df = parse_svo(r)
    arr = pd.concat([arr, df]).reset_index().drop(["index"], axis=1)
    

2022-04-29
Arrived SU 6236 FV
Cancelled SU 6484 SU
Arrived SU 0031 SU
Cancelled SU 6262 SU
Arrived SU 1129 SU
Arrived SU 1007 SU
Arrived SU 1383 SU
Cancelled SU 6496 SU
Arrived SU 6712 FV
Arrived SU 1303 SU
Arrived SU 6536 FV
Cancelled SU 6400 SU
Arrived SU 6040 FV
Arrived SU 6408 FV
Arrived SU 1327 SU
Cancelled SU 6488 SU
Arrived SU 1369 SU
Arrived SU 1245 SU
Cancelled SU 6276 SU
Cancelled SU 6348 SU
Arrived SU 0035 SU
Arrived SU 1763 SU
Arrived SU 1429 SU
Arrived SU 1009 SU
Arrived SU 6248 FV
Arrived SU 1395 SU
Arrived SU 1261 SU
Arrived SU 0033 SU
Arrived SU 1899 SU
Arrived SU 1217 SU
Arrived SU 1503 SU
Cancelled SU 6468 SU
Arrived SU 1407 SU
Arrived SU 1335 SU
Arrived SU 6302 FV
Arrived SU 1289 SU
Arrived SU 1185 SU
Arrived SU 1481 SU
Arrived SU 1533 SU
Arrived SU 6524 FV
Arrived SU 1883 SU
Cancelled SU 6076 SU
Cancelled SU 6506 SU
Arrived SU 1509 SU
Arrived SU 1199 SU
Cancelled SU 6504 SU
Arrived SU 0047 SU
Arrived SU 1459 SU
Arrived SU 1437 SU
Arrived SU 6382 FV
Arrived SU 1451 S

Arrived SU 6712 FV
Arrived SU 1771 SU
2022-05-01
Arrived SU 6236 FV
Cancelled SU 6484 SU
Arrived SU 0031 SU
Cancelled SU 6262 SU
Arrived SU 1785 SU
Cancelled SU 6296 SU
Arrived SU 1017 SU
Arrived SU 6712 FV
Cancelled SU 6496 SU
Arrived SU 1773 SU
Arrived SU 6536 FV
Arrived SU 6400 FV
Arrived SU 6040 FV
Arrived SU 1119 SU
Arrived SU 6408 FV
Arrived SU 1327 SU
Cancelled SU 6488 SU
Arrived SU 6276 FV
Arrived SU 1511 SU
Arrived SU 1503 SU
Arrived SU 1009 SU
Arrived SU 1245 SU
Arrived SU 1429 SU
Arrived SU 1763 SU
Arrived SU 6248 FV
Arrived SU 1395 SU
Arrived SU 1647 SU
Arrived SU 1261 SU
Arrived SU 1441 SU
Arrived SU 1217 SU
Arrived SU 0033 SU
Arrived SU 1481 SU
Arrived SU 6312 FV
Cancelled SU 6468 SU
Arrived SU 6524 FV
Arrived SU 1185 SU
Arrived SU 1335 SU
Landed SU 1289 SU
Arrived SU 1533 SU
Arrived SU 1509 SU
Cancelled SU 6506 SU
Arrived SU 1383 SU
Arrived SU 0513 SU
Arrived SU 1199 SU
Cancelled SU 6504 SU
Arrived SU 0047 SU
Arrived SU 1967 SU
Arrived SU 1437 SU
Arrived SU 1459 SU
Arriv

Cancelled SU 6430 SU
Arrived SU 6532 FV
Arrived SU 0011 SU
Arrived SU 1731 SU
Arrived SU 1269 SU
Arrived SU 1483 SU
Arrived SU 6284 FV
Arrived SU 1189 SU
Cancelled SU 6466 SU
Arrived SU 1831 SU
Arrived SU 1719 SU
Arrived SU 1001 SU
Cancelled SU 6288 SU
Arrived SU 6378 FV
Arrived SU 6258 FV
Arrived SU 1211 SU
Arrived SU 1333 SU
Arrived SU 1701 SU
Arrived SU 1321 SU
Arrived SU 1221 SU
Cancelled SU 6478 SU
Cancelled SU 6338 SU
Arrived SU 1191 SU
Cancelled SU 6226 SU
Arrived SU 6546 FV
Arrived SU 1391 SU
Arrived SU 1201 SU
Arrived SU 1015 SU
Arrived SU 6434 FV
Cancelled SU 6406 SU
Arrived SU 1705 SU
Arrived SU 0015 SU
Arrived SU 6290 FV
Cancelled SU 6458 SU
Arrived SU 1781 SU
Cancelled SU 6240 SU
Arrived SU 1231 SU
Cancelled SU 6500 SU
Arrived SU 1563 SU
Arrived SU 1759 SU
Arrived SU 1715 SU
Arrived SU 0017 SU
Cancelled SU 6068 SU
Cancelled SU 6362 SU
Arrived SU 1139 SU
Arrived SU 1417 SU
Arrived SU 6042 FV
Arrived SU 1703 SU
Cancelled SU 6528 SU
Arrived SU 1463 SU
Arrived SU 1053 SU
Arriv

Cancelled SU 6512 SU
Cancelled SU 6142 SU
Cancelled SU 6272 SU
Arrived SU 1833 SU
Arrived SU 1461 SU
Arrived SU 6230 FV
Arrived SU 1313 SU
Arrived SU 1117 SU
Arrived SU 1139 SU
Arrived SU 0023 SU
Arrived SU 1643 SU
Arrived SU 1681 SU
Arrived SU 1413 SU
Arrived SU 1135 SU
Arrived SU 1137 SU
Arrived SU 6718 FV
Arrived SU 0289 SU
Arrived SU 1233 SU
Arrived SU 1005 SU
Arrived SU 1487 SU
Arrived SU 1423 SU
Arrived SU 1183 SU
Arrived SU 0039 SU
Arrived SU 1193 SU
Arrived SU 6530 FV
Arrived SU 1769 SU
Arrived SU 6604 FV
Arrived SU 1123 SU
Cancelled SU 6212 SU
Cancelled SU 6470 SU
Arrived SU 0025 SU
Arrived SU 1507 SU
Arrived SU 1331 SU
Arrived SU 1325 SU
Arrived SU 6492 FV
Arrived SU 1435 SU
Cancelled SU 6394 SU
Arrived SU 1405 SU
Cancelled SU 6336 SU
Scheduled SU 1249 SU
Arrived SU 1215 SU
Arrived SU 1311 SU
Arrived SU 1365 SU
Arrived SU 1175 SU
Arrived SU 6380 FV
Arrived SU 6350 FV
Cancelled SU 6070 SU
Arrived SU 6608 FV
Arrived SU 1079 SU
Arrived SU 1177 SU
Arrived SU 0027 SU
Arrived SU 11

Arrived SU 1125 SU
Arrived SU 1177 SU
Arrived SU 1079 SU
Arrived SU 6422 FV
Arrived SU 0027 SU
Arrived SU 6608 FV
Arrived SU 1833 SU
Arrived SU 1011 SU
Arrived SU 6712 FV
Arrived SU 1243 SU
Cancelled SU 6252 SU
Arrived SU 1059 SU
Arrived SU 1415 SU
Arrived SU 1129 SU
Arrived SU 1369 SU
Arrived SU 0029 SU
Arrived SU 1195 SU
Arrived SU 6262 FV
Cancelled SU 6994 SU
Arrived SU 6484 FV
2022-05-08
Arrived SU 6262 FV
Arrived SU 6484 FV
Cancelled SU 6236 SU
Arrived SU 0031 SU
Arrived SU 1007 SU
Cancelled SU 6496 SU
Arrived SU 1303 SU
Arrived SU 6536 FV
Cancelled SU 6400 SU
Arrived SU 1119 SU
Arrived SU 6040 FV
Arrived SU 6408 FV
Arrived SU 1327 SU
Cancelled SU 6488 SU
Arrived SU 1409 SU
Arrived SU 1245 SU
Arrived SU 1131 SU
Arrived SU 6276 FV
Arrived SU 6348 FV
Arrived SU 1549 SU
Arrived SU 1503 SU
Arrived SU 1511 SU
Arrived SU 0035 SU
Arrived SU 1429 SU
Arrived SU 1009 SU
Arrived SU 1763 SU
Arrived SU 1395 SU
Cancelled SU 6248 SU
Arrived SU 1441 SU
Arrived SU 1647 SU
Arrived SU 1261 SU
Arrive

...

Arrived SU 1611 SU
Arrived SU 1313 SU
Cancelled SU 6230 SU
Arrived SU 1069 SU
Arrived SU 0023 SU
Arrived SU 1643 SU
Arrived SU 1137 SU
Arrived SU 6604 FV
Cancelled SU 6278 SU
Arrived SU 1681 SU
Arrived SU 1291 SU
Arrived SU 1413 SU
Arrived SU 1363 SU
Arrived SU 1519 SU
Arrived SU 2143 SU
Arrived SU 1233 SU
Arrived SU 1183 SU
Arrived SU 1487 SU
Arrived SU 1005 SU
Arrived SU 1123 SU
Arrived SU 6540 FV
Arrived SU 1423 SU
Arrived SU 1893 SU
Arrived SU 1193 SU
Arrived SU 1769 SU
Arrived SU 1553 SU
Cancelled SU 6212 SU
Cancelled SU 6196 SU
Cancelled SU 6470 SU
Arrived SU 6908 FV
Arrived SU 1507 SU
Arrived SU 1385 SU
Arrived SU 1457 SU
Arrived SU 6394 FV
Arrived SU 2137 SU
Arrived SU 1325 SU
Arrived SU 0025 SU
Arrived SU 1359 SU
Arrived SU 1697 SU
Cancelled SU 6360 SU
Cancelled SU 6492 SU
Arrived SU 0039 SU
Arrived SU 1341 SU
Cancelled SU 6336 SU
Arrived SU 1259 SU
Arrived SU 1013 SU
Arrived SU 1405 SU
Arrived SU 1365 SU
Arrived SU 1311 SU
Arrived SU 1215 SU
Arrived SU 1293 SU
Arrived SU 1489

Arrived SU 1495 SU
Arrived SU 1731 SU
Arrived SU 1001 SU
Arrived SU 1297 SU
Cancelled SU 6466 SU
Arrived SU 1269 SU
Cancelled SU 6288 SU
Arrived SU 1411 SU
Arrived SU 1831 SU
Arrived SU 1211 SU
Cancelled SU 6310 SU
Arrived SU 0065 SU
Arrived SU 1771 SU
Arrived SU 1319 SU
Cancelled SU 6258 SU
Cancelled SU 6378 FV
Landed SU 1333 SU
Cancelled SU 6478 SU
Arrived SU 1221 SU
Cancelled SU 6338 SU
Arrived SU 1545 SU
Landed SU 1191 SU
Cancelled SU 6226 SU
Cancelled SU 6546 SU
Arrived SU 0013 SU
Arrived SU 1759 SU
Arrived SU 1201 SU
Arrived SU 1705 SU
Arrived SU 1015 SU
Arrived SU 0233 SU
Arrived SU 1641 SU
Arrived SU 1517 SU
Cancelled SU 6458 SU
Arrived SU 1679 SU
Arrived SU 0015 SU
Cancelled SU 6240 SU
Arrived SU 2149 SU
Arrived SU 1401 SU
Arrived SU 6220 FV
Arrived SU 1715 SU
Cancelled SU 6500 SU
Cancelled SU 6434 SU
Arrived SU 1669 SU
Arrived SU 1231 SU
Arrived SU 6416 FV
Arrived SU 1219 SU
Arrived SU 6992 FV
Arrived SU 6076 FV
Arrived SU 1593 SU
Arrived SU 6362 FV
Arrived SU 0017 SU
Arrived

Cancelled SU 6252 SU
Arrived SU 0069 SU
Arrived SU 2147 SU
Arrived SU 1301 SU
Arrived SU 1133 SU
Arrived SU 1369 SU
Arrived SU 1415 SU
Arrived SU 1605 SU
Arrived SU 6558 FV
Arrived SU 1195 SU
Arrived SU 1129 SU
Cancelled SU 6256 SU
Arrived SU 0029 SU
Arrived SU 1003 SU
Arrived SU 6712 FV
Arrived SU 1317 SU
Arrived SU 6236 FV
Arrived SU 6390 FV
Arrived SU 6262 FV
Arrived SU 6994 FV
2022-06-29
Arrived SU 1317 SU
Arrived SU 6236 FV
Arrived SU 6262 FV
Cancelled SU 6484 SU
Arrived SU 6994 FV
Arrived SU 0031 SU
Arrived SU 1063 SU
Arrived SU 1007 SU
Cancelled SU 6496 SU
Arrived SU 1303 SU
Arrived SU 1419 SU
Arrived SU 0045 SU
Arrived SU 6536 FV
Cancelled SU 6400 SU
Cancelled SU 6156 SU
Arrived SU 2145 SU
Arrived SU 1131 SU
Arrived SU 0063 SU
Arrived SU 1119 SU
Arrived SU 6040 FV
Arrived SU 6616 FV
Arrived SU 1397 SU
Arrived SU 1197 SU
Arrived SU 2135 SU
Cancelled SU 6408 SU
Arrived SU 0037 SU
Arrived SU 6488 FV
Arrived SU 6602 FV
Arrived SU 1409 SU
Arrived SU 1549 SU
Arrived SU 1511 SU
Arrive

Cancelled SU 6490 SU
Cancelled SU 6124 SU
Arrived SU 1483 SU
Arrived SU 2139 SU
Arrived SU 1255 SU
Cancelled SU 6430 FV
Arrived SU 1719 SU
Arrived SU 1189 SU
Cancelled SU 6300 SU
Arrived SU 1391 SU
Arrived SU 6532 FV
Arrived SU 0011 SU
Arrived SU 6284 FV
Cancelled SU 6406 SU
Arrived SU 1269 SU
Arrived SU 1525 SU
Arrived SU 1731 SU
Arrived SU 1297 SU
Arrived SU 1771 SU
Cancelled SU 6466 SU
Arrived SU 1411 SU
Cancelled SU 6288 SU
Arrived SU 0065 SU
Arrived SU 1831 SU
Arrived SU 1211 SU
Cancelled SU 6310 SU
Cancelled SU 6258 SU
Cancelled SU 6378 FV
Arrived SU 1333 SU
Landed SU 1221 SU
Arrived SU 1319 SU
Cancelled SU 6478 SU
Cancelled SU 6338 SU
Arrived SU 1001 SU
Arrived SU 1705 SU
Arrived SU 1545 SU
Arrived SU 1191 SU
Cancelled SU 6546 SU
Cancelled SU 6226 SU
Arrived SU 1759 SU
Arrived SU 1201 SU
Arrived SU 0013 SU
Arrived SU 1781 SU
Arrived SU 6290 FV
Arrived SU 1517 SU
Arrived SU 2149 SU
Arrived SU 1015 SU
Arrived SU 1715 SU
Arrived SU 1679 SU
Cancelled SU 6458 SU
Arrived SU 1563 SU
Ar

Arrived SU 1123 SU
Arrived SU 1183 SU
Arrived SU 1005 SU
Arrived SU 1423 SU
Arrived SU 1193 SU
Arrived SU 0039 SU
Arrived SU 1445 SU
Arrived SU 1769 SU
Arrived SU 1553 SU
Arrived SU 1385 SU
Cancelled SU 6212 SU
Cancelled SU 6470 SU
Arrived SU 1457 SU
Arrived SU 1025 SU
Arrived SU 2137 SU
Arrived SU 6538 FV
Arrived SU 1507 SU
Arrived SU 1435 SU
Arrived SU 6718 FV
Arrived SU 0025 SU
Arrived SU 1359 SU
Arrived SU 1259 SU
Cancelled SU 6360 SU
Arrived SU 1325 SU
Cancelled SU 6394 SU
Cancelled SU 6492 SU
Arrived SU 1065 SU
Arrived SU 1405 SU
Cancelled SU 6336 SU
Arrived SU 1341 SU
Arrived SU 1013 SU
Arrived SU 1697 SU
Arrived SU 1365 SU
Scheduled SU 1991 SU
Arrived SU 1489 SU
Arrived SU 0061 SU
Arrived SU 1991 SU
Arrived SU 1215 SU
Cancelled SU 6070 SU
Arrived SU 1717 SU
Arrived SU 0055 SU
Arrived SU 1311 SU
Arrived SU 6350 FV
Cancelled SU 6380 SU
Arrived SU 1125 SU
Arrived SU 1059 SU
Arrived SU 1175 SU
Cancelled SU 6422 SU
Arrived SU 6606 FV
Arrived SU 1137 SU
Arrived SU 6608 FV
Arrived SU 

In [73]:
arr

,carrier,flight_number,aircraft_type,aircraft_registration,registration_start,registration_end,boarding_start,boarding_end,departure_airportCode,departure_actualBlockOff,departure_scheduledDeparture,departure_actualTakeOff,arrival_gate,arrival_scheduled,arrival_latest,arrival_terminal,arrival_scheduledArrival,arrival_actualTouchDown,arrival_deboardingStart,arrival_deboardingStop,deboarding_start,deboarding_end,status,operatingBy
0,SU,6236,SU9,89113,2022-04-27T19:30:00Z,2022-04-28T18:45:00Z,2022-04-28T18:42:00Z,2022-04-28T19:02:00Z,GOJ,2022-04-28T19:22:00Z,2022-04-28T19:30:00Z,2022-04-28T19:31:00Z,NaN,SVO,SVO,B,2022-04-28T21:05:00Z,2022-04-28T20:28:00Z,NaN,NaN,2022-04-28T20:37:00Z,2022-04-28T21:07:00Z,Arrived,FV
1,SU,0031,73H,73109,2022-04-27T19:55:00Z,2022-04-28T19:10:00Z,2022-04-28T19:10:00Z,2022-04-28T19:30:00Z,LED,2022-04-28T19:50:00Z,2022-04-28T19:55:00Z,2022-04-28T20:10:00Z,1,SVO,SVO,B,2022-04-28T21:25:00Z,2022-04-28T21:02:00Z,NaN,NaN,2022-04-28T21:07:00Z,2022-04-28T21:37:00Z,Arrived,SU
2,SU,1129,321,VPBTR,2022-04-27T17:20:00Z,2022-04-28T16:35:00Z,2022-04-28T16:50:00Z,2022-04-28T17:10:00Z,AER,2022-04-28T17:30:00Z,2022-04-28T16:35:00Z,2022-04-28T17:42:00Z,110,SVO,SVO,B,2022-04-28T20:50:00Z,2022-04-28T21:22:00Z,2022-04-28T21:33:00Z,NaN,2022-04-28T21:33:00Z,2022-04-28T21:50:00Z,Arrived,SU
3,SU,1007,321,73704,2022-04-27T19:00:00Z,2022-04-28T18:15:00Z,2022-04-28T18:19:00Z,2022-04-28T18:39:00Z,KGD,2022-04-28T18:59:00Z,2022-04-28T19:00:00Z,2022-04-28T19:12:00Z,1,SVO,SVO,B,2022-04-28T21:45:00Z,2022-04-28T21:27:00Z,2022-04-28T21:35:00Z,NaN,2022-04-28T21:35:00Z,2022-04-28T21:53:00Z,Arrived,SU
4,SU,1383,320,VPBIY,2022-04-27T18:30:00Z,2022-04-28T17:45:00Z,2022-04-28T17:47:00Z,2022-04-28T18:07:00Z,HMA,2022-04-28T18:27:00Z,2022-04-28T01:55:00Z,2022-04-28T18:38:00Z,NaN,SVO,SVO,B,2022-04-28T05:10:00Z,2022-04-28T21:29:00Z,NaN,NaN,2022-04-28T21:38:00Z,2022-04-28T22:08:00Z,Arrived,SU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13585,SU,6236,SU9,89100,2022-06-30T19:30:00Z,2022-07-01T18:45:00Z,2022-07-01T18:46:00Z,2022-07-01T19:06:00Z,GOJ,2022-07-01T19:26:00Z,2022-07-01T19:30:00Z,2022-07-01T19:35:00Z,NaN,SVO,SVO,B,2022-07-01T21:05:00Z,2022-07-01T20:33:00Z,NaN,NaN,2022-07-01T20:46:00Z,2022-07-01T21:16:00Z,Arrived,FV
13586,SU,1293,320,73168,2022-06-30T18:40:00Z,2022-07-01T17:55:00Z,2022-07-01T18:10:00Z,2022-07-01T18:30:00Z,IJK,2022-07-01T18:50:00Z,2022-07-01T16:40:00Z,2022-07-01T19:00:00Z,119,SVO,SVO,B,2022-07-01T18:50:00Z,2022-07-01T20:44:00Z,2022-07-01T20:53:00Z,NaN,2022-07-01T20:53:00Z,2022-07-01T21:03:00Z,Arrived,SU
13587,SU,1317,321,73712,2022-06-30T18:35:00Z,2022-07-01T17:50:00Z,2022-07-01T17:51:00Z,2022-07-01T18:11:00Z,MMK,2022-07-01T18:31:00Z,2022-07-01T18:35:00Z,2022-07-01T18:45:00Z,108,SVO,SVO,B,2022-07-01T21:05:00Z,2022-07-01T20:49:00Z,2022-07-01T21:03:00Z,NaN,2022-07-01T21:03:00Z,2022-07-01T21:09:00Z,Arrived,SU
13588,SU,1133,73H,73103,2022-06-30T16:50:00Z,2022-07-01T16:05:00Z,2022-07-01T16:34:00Z,2022-07-01T16:54:00Z,AER,2022-07-01T17:14:00Z,2022-07-01T16:10:00Z,2022-07-01T17:30:00Z,120,SVO,SVO,B,2022-07-01T20:10:00Z,2022-07-01T20:55:00Z,NaN,NaN,2022-07-01T21:06:00Z,2022-07-01T21:36:00Z,Arrived,SU


In [74]:
arr = arr.sort_values(by='arrival_scheduledArrival')

In [75]:
arr = arr.drop_duplicates(['flight_number', 'arrival_scheduledArrival'])

In [76]:
arr.to_csv("svo_arr_aeroflot_rossiya_05_06_2022.csv", index=False)

In [85]:
arr.departure_airportCode.value_counts()

LED    1304
AER    1047
SVX     617
KGD     540
KZN     430
       ... 
CMB      16
DYR      16
IKU       5
SVO       1
NAL       1
Name: departure_airportCode, Length: 77, dtype: int64